In [1]:
import findspark
findspark.init('/home/osboxes/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [2]:
myPath = os.path.join('/home', 'osboxes', 'CourseMaterial', 'Spark_for_Machine_Learning', 'Logistic_Regression')

print(myPath)

os.chdir(myPath)

/home/osboxes/CourseMaterial/Spark_for_Machine_Learning/Logistic_Regression


In [3]:
os.listdir()

['Titanic_Log_Regression_Code_Along.ipynb',
 'new_customers.csv',
 '.ipynb_checkpoints',
 'titanic.csv',
 'sample_libsvm_data.txt',
 'Logistic_Regression_Consulting_Project_SOLUTIONS.ipynb',
 'Logistic_Regression_Example.ipynb',
 'Logistic_Regression_Consulting_Project.ipynb',
 'customer_churn.csv']

In [4]:
# set up the session info

from pyspark.sql import SparkSession

name = 'churn'

spark = SparkSession.builder.appName(name).getOrCreate()

from pyspark.ml.classification import LogisticRegression

In [5]:
# get the churn data

df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [6]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [9]:
# get the columns for selection
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [10]:
# get the assembler

from pyspark.ml.feature import VectorAssembler


In [11]:
# assemble the output col of features
# do not need string data

assembler = VectorAssembler(inputCols=['Age',
                                       'Total_Purchase',
                                       'Account_Manager',
                                       'Years',
                                       'Num_Sites'],outputCol='features')

In [12]:
# use the assembler to transform entire data set

output = assembler.transform(df)

In [13]:
# select the 2 columns used by spark
final_data = output.select('features', 'churn')
final_data.head()

Row(features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]), churn=1)

In [14]:
train, test = final_data.randomSplit([0.7, 0.3])

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
# create object

lr_churn = LogisticRegression(labelCol='churn')

In [17]:
# fit the data

fitted_churn = lr_churn.fit(train)

In [18]:
training_sum = fitted_churn.summary

In [19]:
training_sum.predictions.describe().show()

+-------+-------------------+-----------------+
|summary|              churn|       prediction|
+-------+-------------------+-----------------+
|  count|                656|              656|
|   mean|0.16615853658536586|            0.125|
| stddev| 0.3725069178356457|0.330971274785025|
|    min|                0.0|              0.0|
|    max|                1.0|              1.0|
+-------+-------------------+-----------------+



In [20]:
# evaluate the model performance

from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
pred_and_labels = fitted_churn.evaluate(test)

In [22]:
# get a view of the performance
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.65120109361873...|[0.65728107530969...|       0.0|
|[29.0,10203.18,1....|    0|[3.81265761684047...|[0.97838800028433...|       0.0|
|[29.0,13240.01,1....|    0|[6.67356550204794...|[0.99873771106166...|       0.0|
|[29.0,13255.05,1....|    0|[4.13025072540097...|[0.98417559130355...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.82225948996883...|[0.99704783365103...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.30413962097747...|[0.98666764708536...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.13732772478832...|[0.95840648486671...|       0.0|
|[30.0,10183.98,1....|    0|[2.87596428111564...|[0.94664539498737...|       0.0|
|[30.0,10744.14,1....|    1|[1.89602979972145...|[0.86944151707413...|       0.0|
|[30.0,10960.52,

In [23]:
# AUC
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [24]:
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

0.7779046017061156

In [25]:
# save the final model
# fit on both test and train

final_lr_model = lr_churn.fit(final_data)

In [26]:
# now test on new data

new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [27]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [28]:
# use the assembler on the new data

test_new_customers = assembler.transform(new_customers)

In [30]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [31]:
final_results = final_lr_model.transform(test_new_customers)
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168705251440...|[0.90218018099704...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207530595020...|[0.00198380445828...|       

In [32]:
# just get the columns that we need
# those with 1's are likely to leave

final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

